In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip

In [1]:
!python3 preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


In [2]:
!python3 preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

In [ ]:
pip install torch

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
pip install torch==2.0.1+cu117 torchvision==0.15.2+cu117 torchaudio==2.0.2+cu117 --index-url https://download.pytorch.org/whl/cu117

In [ ]:
pip install scikit-learn

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle as cp
from sklearn.metrics import f1_score
import time
#from sliding_window import sliding_window

In [5]:
# Updated sliding window function
def sliding_window(a, ws, ss=None, flatten=True):
    if ss is None:
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)

    ws = np.array(ws)
    ss = np.array(ss)
    shape = np.array(a.shape)

    ls = [len(shape), len(ws), len(ss)]
    if 1 != len(set(ls)):
        raise ValueError('a.shape, ws and ss must all have the same length. They were %s' % str(ls))

    if np.any(ws > shape):
        raise ValueError('ws cannot be larger than a in any dimension. a.shape was %s and ws was %s' % (str(a.shape), str(ws)))

    newshape = norm_shape(((shape - ws) // ss) + 1)
    newshape += norm_shape(ws)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides
    strided = np.lib.stride_tricks.as_strided(a, shape=newshape, strides=newstrides)
    if not flatten:
        return strided

    meat = len(ws) if ws.shape else 0
    firstdim = (np.prod(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + newshape[-meat:]
    dim = tuple(filter(lambda i: i != 1, dim))
    return strided.reshape(dim)

def norm_shape(shape):
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        pass

    try:
        t = tuple(shape)
        return t
    except TypeError:
        pass

    raise TypeError('shape must be an int, or a tuple of ints')

# Hardcoded parameters
NB_SENSOR_CHANNELS = 113
NUM_CLASSES = 18
SLIDING_WINDOW_LENGTH = 24
SLIDING_WINDOW_STEP = 12
BATCH_SIZE = 16
NUM_FILTERS = 64
FILTER_SIZE = 5
NUM_UNITS_LSTM = 128
LEARNING_RATE = 0.0001  # Reduced learning rate
NUM_EPOCHS = 100


In [6]:
# Load dataset
def load_dataset(filename):
    with open(filename, 'rb') as f:
        data = cp.load(f)
    X_train, y_train = data[0]
    X_test, y_test = data[1]
    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {}, test {}".format(X_train.shape, X_test.shape))
    return X_train.astype(np.float32), y_train.astype(np.uint8), X_test.astype(np.float32), y_test.astype(np.uint8)

# Apply sliding window
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x, (ws, data_x.shape[1]), (ss, 1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y, ws, ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Normalize data
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return (data - mean) / std



In [7]:
# Define the network
class DeepConvLSTM(nn.Module):
    def __init__(self):
        super(DeepConvLSTM, self).__init__()
        self.conv1 = nn.Conv2d(1, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv2 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv3 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv4 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.lstm1 = nn.LSTM(NUM_FILTERS * 113, NUM_UNITS_LSTM, batch_first=True)
        self.lstm2 = nn.LSTM(NUM_UNITS_LSTM, NUM_UNITS_LSTM, batch_first=True)
        self.fc = nn.Linear(NUM_UNITS_LSTM, NUM_CLASSES)

        # Weight initialization
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv4.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = x.permute(0, 2, 1, 3).contiguous().view(x.size(0), x.size(2), -1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.fc(x[:, -1, :])
        return x


In [8]:
# Load data
print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

# Normalize data
X_train = normalize(X_train)
X_test = normalize(X_test)

# Apply sliding window
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {}, targets {}".format(X_train.shape, y_train.shape))
print(" ..after sliding window (testing): inputs {}, targets {}".format(X_test.shape, y_test.shape))

# Reshape data
X_train = X_train.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
X_test = X_test.reshape((-1, 1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))

# Create datasets and dataloaders
train_data = torch.utils.data.TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test_data = torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# Initialize the model, loss function and optimizer
model = DeepConvLSTM().to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)


/tmp/ipykernel_115940/2801352964.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  i = int(shape)


 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [ ]:

# Training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {train_loss / len(train_loader)}")




In [ ]:
torch.save(model.state_dict(), 'weights/DeepConvLSTM_trained_oppChallenge_gestures.pth')

In [9]:
class DeepConvLSTM(nn.Module):
    def __init__(self):
        super(DeepConvLSTM, self).__init__()
        self.conv1 = nn.Conv2d(1, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv2 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv3 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.conv4 = nn.Conv2d(NUM_FILTERS, NUM_FILTERS, (FILTER_SIZE, 1))
        self.lstm1 = nn.LSTM(NUM_FILTERS * 113, NUM_UNITS_LSTM, batch_first=True)
        self.lstm2 = nn.LSTM(NUM_UNITS_LSTM, NUM_UNITS_LSTM, batch_first=True)
        self.fc = nn.Linear(NUM_UNITS_LSTM, NUM_CLASSES)

        # Weight initialization
        nn.init.kaiming_uniform_(self.conv1.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv2.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv3.weight, nonlinearity='relu')
        nn.init.kaiming_uniform_(self.conv4.weight, nonlinearity='relu')
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = x.permute(0, 2, 1, 3).contiguous().view(x.size(0), x.size(2), -1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.fc(x[:, -1, :])
        return x

model = DeepConvLSTM()
model.load_state_dict(torch.load('weights/DeepConvLSTM_trained_oppChallenge_gestures.pth'))


<All keys matched successfully>

In [10]:
model.to(torch.device('cpu'))

DeepConvLSTM(
  (conv1): Conv2d(1, 64, kernel_size=(5, 1), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(5, 1), stride=(1, 1))
  (lstm1): LSTM(7232, 128, batch_first=True)
  (lstm2): LSTM(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=18, bias=True)
)

In [14]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Evaluation
model.eval()
test_pred = []
test_true = []
batch_times = []
sample_times = []

with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to('cpu'), targets.to('cpu')

        # Start time for the batch
        start_time = time.time()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # End time for the batch
        end_time = time.time()

        # Record the inference time for the batch
        batch_inference_time = end_time - start_time
        batch_times.append(batch_inference_time)

        # Calculate per-sample time for the batch
        batch_size = inputs.size(0)
        per_sample_time = batch_inference_time / batch_size
        sample_times.append(per_sample_time)

        test_pred.extend(preds.cpu().numpy())
        test_true.extend(targets.cpu().numpy())

        # Print per-batch time and per-sample time for the current batch
        #print(f"Batch {batch_idx + 1}:")
        #print(f"  Batch Inference Time: {batch_inference_time:.4f} seconds")
        #print(f"  Per-Sample Inference Time: {per_sample_time:.8f} seconds")

# Calculate metrics
accuracy = accuracy_score(test_true, test_pred)
macro_precision = precision_score(test_true, test_pred, average='macro')
macro_recall = recall_score(test_true, test_pred, average='macro')
macro_f1 = f1_score(test_true, test_pred, average='macro')

# Calculate average batch time and average per-sample time
average_batch_time = sum(batch_times) / len(batch_times)
average_per_sample_time = sum(sample_times) / len(sample_times)

# Results presentation
print(f"\nOverall Test Results:")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")
print(f"Average Batch Inference Time: {average_batch_time:.4f} seconds")
print(f"Average Per-Sample Inference Time: {average_per_sample_time:.8f} seconds")



Overall Test Results:
Test Accuracy: 0.8792
Macro Precision: 0.6317
Macro Recall: 0.5423
Macro F1-score: 0.5617
Average Batch Inference Time: 0.0049 seconds
Average Per-Sample Inference Time: 0.00030426 seconds
